In [228]:
import praw
import urllib.request
import certifi, ssl
import xmltodict
from datetime import datetime

from Corpus import Corpus
from RedditDocument import RedditDocument
from ArxivDocument import ArxivDocument
from SearchEngine import SearchEngine

import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

CREATION CORPUS

In [229]:
corpus = Corpus("Corpus_Reddit_Arxiv")


RECUPERATION DONNEES REDDIT

In [230]:
reddit = praw.Reddit(
    client_id="vc2Hro3ys8p9rqCG6bHeAg",
    client_secret="gedHKB0SBBkm9H2vEwhnsCPw5UykVg",
    user_agent="WebScraping"
)

ml_subreddit = reddit.subreddit("MachineLearning")

for post in ml_subreddit.hot(limit=50):
    titre = post.title.replace("\n", " ")
    auteur = str(post.author) if post.author else "Inconnu"
    date_pub = datetime.fromtimestamp(post.created_utc)
    url = f"https://www.reddit.com{post.permalink}"
    texte = post.selftext if post.selftext else post.title
    nb_comments = post.num_comments

    doc = RedditDocument(titre, auteur, date_pub, url, texte, nb_comments)
    corpus.add_document(doc)


RECUPERATION DONNEES ARXIV

In [231]:
context = ssl.create_default_context(cafile=certifi.where())
url = "http://export.arxiv.org/api/query?search_query=all:machine&start=0&max_results=50"

data = urllib.request.urlopen(url, context=context).read().decode("utf-8")
dict_data = xmltodict.parse(data)
entries = dict_data["feed"]["entry"]

for entry in entries:
    titre = entry["title"].replace("\n", " ")

    if isinstance(entry["author"], list):
        auteur = entry["author"][0]["name"]
        coauthors = [a["name"] for a in entry["author"][1:]]
    else:
        auteur = entry["author"]["name"]
        coauthors = []

    date_pub = datetime.fromisoformat(entry["published"].replace("Z", "+00:00"))
    url = entry["id"]
    texte = entry["summary"].replace("\n", " ")

    doc = ArxivDocument(titre, auteur, date_pub, url, texte, coauthors)
    corpus.add_document(doc)


INSTANCE MOTEUR DE RECHERCHE

In [232]:
engine = SearchEngine(corpus)
print("✅ Corpus Reddit + ArXiv chargé")


✅ Corpus Reddit + ArXiv chargé


In [233]:
import ipywidgets as widgets
from IPython.display import display

In [234]:
# Widgets de recherche
query_box = widgets.Text(description="Mots clés:")

# Source / corpus
corpus_options = ['Tous', 'Reddit', 'ArXiv']
source_dropdown = widgets.Dropdown(options=corpus_options, value='Tous', description='Source:')

# Auteur
author_box = widgets.Text(description="Auteur:", placeholder="Nom de l'auteur")

# Période
date_start = widgets.DatePicker(description='Date début:')
date_end = widgets.DatePicker(description='Date fin:')

# Top N résultats
top_slider = widgets.IntSlider(value=5, min=1, max=20, description="Top N:")

# Bouton recherche et bouton graphique
search_button = widgets.Button(description="Rechercher", button_style='success')
plot_button = widgets.Button(description="Évolution temporelle", button_style='info')

# Zone de sortie
output = widgets.Output()

# Layout
ui = widgets.VBox([
    query_box,
    widgets.HBox([source_dropdown, author_box, top_slider]),
    widgets.HBox([date_start, date_end]),
    widgets.HBox([search_button, plot_button]),
    output
])

In [235]:
def search_button_clicked(b):
    """
    Fonction déclenchée lorsque l'utilisateur clique sur le bouton de recherche.
    Elle effectue une recherche TF-IDF sur le corpus filtré selon :
    - source
    - auteur
    - période
    et affiche les résultats avec score.
    """

    # Affichage dans le widget output (Jupyter / ipywidgets)
    with output:
        # Efface l'affichage précédent
        output.clear_output()

        # Récupération des paramètres saisis par l'utilisateur
        query = query_box.value.strip()       # mot-clé à rechercher
        source = source_dropdown.value        # type de document (Arxiv, Reddit...)
        author = author_box.value.strip()     # auteur
        start = date_start.value              # date début
        end = date_end.value                  # date fin
        top_n = top_slider.value              # nombre de résultats à afficher

        # Vérification que la requête n'est pas vide
        if not query:
            print("⚠️ Veuillez entrer un mot-clé")
            return

        # Filtrage des documents selon les critères
        filtered_docs = filter_documents(corpus, source, author, start, end)
        if not filtered_docs:
            print("⚠️ Aucun document ne correspond aux filtres")
            return

        # Création d'un corpus temporaire uniquement avec les documents filtrés
        corpus_temp = Corpus("temp")
        for d in filtered_docs:
            corpus_temp.add_document(d)

        # Initialisation du moteur de recherche sur le sous-corpus
        engine_sub = SearchEngine(corpus_temp)

        # Recherche TF-IDF
        results = engine_sub.search(query, k=top_n)

        # Vérification des résultats
        if not results:
            print("Aucun résultat trouvé")
        else:
            print(f"--- Top {top_n} résultats pour '{query}' ---")
            for i, doc in enumerate(results, 1):
                print(f"{i}. {doc.get('titre','N/A')} ({doc.get('auteur','N/A')}) - {doc.get('date','N/A')}")
                print(f"URL: {doc.get('url','N/A')}")
                print(f"Score TF-IDF: {doc.get('score',0):.3f}")
                print("-"*50)


In [236]:
def plot_button_clicked(b):
    """
    Fonction déclenchée lors du clic sur le bouton de recherche / plot.
    Elle génère une frise temporelle des occurrences d'un mot-clé
    dans les documents filtrés selon l'auteur, la source et la période.
    """

    # Affichage dans un widget de sortie (Jupyter / ipywidgets)
    with output:
        # Effacer la sortie précédente
        output.clear_output()

        # Récupérer la requête saisie par l'utilisateur
        query = query_box.value.strip()
        if not query:
            print("⚠️ Veuillez entrer un mot-clé")
            return

        # Filtrer les documents selon les critères sélectionnés
        filtered_docs = filter_documents(
            corpus,
            source_dropdown.value,   # Filtrer par type de source (ex: Arxiv, Reddit)
            author_box.value.strip(), # Filtrer par auteur
            date_start.value,        # Date de début
            date_end.value           # Date de fin
        )

        # Vérification si aucun document ne correspond aux filtres
        if not filtered_docs:
            print("⚠️ Aucun document ne correspond aux filtres")
            return

        # Construire la frise temporelle
        dates = []
        for d in filtered_docs:
            if d.date:
                try:
                    # Conversion de la date en objet datetime
                    dt = pd.to_datetime(d.date)

                    # Ajouter l'année si le mot-clé est présent dans le texte
                    if query.lower() in d.texte.lower():
                        dates.append(dt.year)
                except:
                    # Ignorer les dates mal formatées
                    continue

        # Vérification si le mot-clé n'apparaît dans aucun document
        if not dates:
            print("Aucun mot trouvé pour cette période")
            return

        # Comptage des occurrences par année
        counts = Counter(dates)
        years = sorted(counts.keys())
        freqs = [counts[y] for y in years]

        # Affichage du graphique
        plt.figure(figsize=(10, 4))
        plt.bar(years, freqs, color='skyblue')
        plt.xlabel("Année")
        plt.ylabel(f"Occurrences de '{query}'")
        plt.title(f"Évolution temporelle du mot '{query}'")
        plt.show()


In [237]:
def filter_documents(corpus, source='Tous', author=None, start=None, end=None):
    """
    Filtre les documents d'un corpus selon plusieurs critères :
    - source : type de document (ex: "Arxiv", "Reddit") ou "Tous"
    - author : nom (ou partie du nom) de l'auteur
    - start : date de début (datetime.date ou compatible)
    - end : date de fin (datetime.date ou compatible)
    
    Retourne une liste de documents correspondant aux filtres.
    """

    # Commence avec tous les documents du corpus
    filtered = list(corpus.documents.values())

    # Filtrage par source si précisé
    if source != 'Tous':
        filtered = [d for d in filtered if d.getType() == source]

    # Filtrage par auteur si précisé
    if author:
        # Recherche insensible à la casse
        filtered = [d for d in filtered if d.auteur and author.lower() in d.auteur.lower()]

    # Filtrage par date de début
    if start:
        filtered = [
            d for d in filtered 
            if d.date and pd.to_datetime(d.date).date() >= start
        ]

    # Filtrage par date de fin
    if end:
        filtered = [
            d for d in filtered 
            if d.date and pd.to_datetime(d.date).date() <= end
        ]

    return filtered


In [238]:
search_button.on_click(search_button_clicked)
plot_button.on_click(plot_button_clicked)


LANCEMENT INTERFACE

In [239]:
display(ui)